In [17]:
from tardis_dev import datasets, get_exchange_details
import logging
import nest_asyncio
nest_asyncio.apply()
# comment out to disable debug logs
logging.basicConfig(level=logging.DEBUG)

# function used by default if not provided via options
def default_file_name(exchange, data_type, date, symbol, format):
    return f"{exchange}_{data_type}_{date.strftime('%Y-%m-%d')}_{symbol}.{format}.gz"


# customized get filename function - saves data in nested directory structure
def file_name_nested(exchange, data_type, date, symbol, format):
    return f"{exchange}/{data_type}/{date.strftime('%Y-%m-%d')}_{symbol}.{format}.gz"


# returns data available at https://api.tardis.dev/v1/exchanges/deribit
deribit_details = get_exchange_details("binance")
# print(deribit_details)

datasets.download(
    # one of https://api.tardis.dev/v1/exchanges with supportsDatasets:true - use 'id' value
    exchange="binance",
    # accepted data types - 'datasets.symbols[].dataTypes' field in https://api.tardis.dev/v1/exchanges/deribit,
    # or get those values from 'deribit_details["datasets"]["symbols][]["dataTypes"] dict above
    data_types=[ "trades", "quotes",  "book_snapshot_25", "book_snapshot_5"],
    # change date ranges as needed to fetch full month or year for example
    from_date="2020-12-01",
    # to date is non inclusive
    to_date="2020-12-02",
    # accepted values: 'datasets.symbols[].id' field in https://api.tardis.dev/v1/exchanges/deribit
    symbols=["BTCUSDT"],
    # (optional) your API key to get access to non sample data as well
    api_key="TD.28DTwLrn3f988WRM.w3hZLmc9mf2kbFO.OrGHMJWJZldog5f.Qob9tqB1fLEbb9l.0-x0-FrbU6jD9Dz.n2vg",
    # (optional) path where data will be downloaded into, default dir is './datasets'
    # download_dir="./datasets",
    # (optional) - one can customize downloaded file name/path (flat dir strucure, or nested etc) - by default function 'default_file_name' is used
    # get_filename=default_file_name,
    # (optional) file_name_nested will download data to nested directory structure (split by exchange and data type)
    # get_filename=file_name_nested,
)

DEBUG:tardis_dev.datasets.download:download started for binance trades BTCUSDT from 2020-12-01 to 2020-12-02
DEBUG:tardis_dev.datasets.download:download finished for binance trades BTCUSDT from 2020-12-01 to 2020-12-02, total time: 6.465068578720093 seconds
DEBUG:tardis_dev.datasets.download:download started for binance quotes BTCUSDT from 2020-12-01 to 2020-12-02
DEBUG:tardis_dev.datasets.download:download finished for binance quotes BTCUSDT from 2020-12-01 to 2020-12-02, total time: 2.8814282417297363 seconds
DEBUG:tardis_dev.datasets.download:download started for binance book_snapshot_25 BTCUSDT from 2020-12-01 to 2020-12-02
DEBUG:tardis_dev.datasets.download:download finished for binance book_snapshot_25 BTCUSDT from 2020-12-01 to 2020-12-02, total time: 7.760175466537476 seconds
DEBUG:tardis_dev.datasets.download:download started for binance book_snapshot_5 BTCUSDT from 2020-12-01 to 2020-12-02
DEBUG:tardis_dev.datasets.download:download finished for binance book_snapshot_5 BTCUSD

In [19]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import random
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]
plt.rcParams["axes.unicode_minus"] = False

class MarketMaker:
    def __init__(self, initial_capital=100000, spread=0.001, order_amount=0.01, 
                 delay_range=(0.1, 0.5), max_inventory=1.0):
        """
        初始化做市商
        :param initial_capital: 初始资金
        :param spread: 买卖价差比例
        :param order_amount: 每次挂单数量
        :param delay_range: 交易延时范围(秒)
        :param max_inventory: 最大持仓限制
        """
        self.capital = initial_capital
        self.inventory = 0.0  # BTC数量
        self.spread = spread
        self.order_amount = order_amount
        self.delay_range = delay_range
        self.max_inventory = max_inventory
        
        # 记录绩效
        self.performance_history = []
        self.inventory_history = []
        self.capital_history = []
        self.timestamps = []
        
        # 订单簿数据
        self.order_book = None
        self.trades = None
        
        # 记录已执行订单
        self.executed_orders = []

    def load_data(self, book_path, trades_path):
        """加载订单簿和交易数据"""
        # 加载订单簿快照
        self.order_book = pd.read_csv(book_path)
        # 转换时间戳
        self.order_book['timestamp'] = pd.to_datetime(self.order_book['timestamp']/1000000)
        
        # 加载交易数据
        self.trades = pd.read_csv(trades_path)
        # 转换时间戳
        self.trades['timestamp'] = pd.to_datetime(self.trades['timestamp']/1000000)
        
        # 按时间排序
        self.order_book.sort_values('timestamp', inplace=True)
        self.trades.sort_values('timestamp', inplace=True)
        
        print(f"加载完成 - 订单簿快照数量: {len(self.order_book)}, 交易记录数量: {len(self.trades)}")

    def get_mid_price(self, book_snapshot):
        """从订单簿快照获取中间价"""
        # 假设数据中有bid_price_0和ask_price_0字段表示最优买卖价
        if 'bids[0].price' in book_snapshot and 'asks[0].price' in book_snapshot:
            return (book_snapshot['bids[0].price'] + book_snapshot['asks[0].price']) / 2
        return None

    def generate_orders(self, book_snapshot):
        """根据当前订单簿生成做市订单"""
        mid_price = self.get_mid_price(book_snapshot)
        if mid_price is None:
            return None
            
        # 根据中间价和价差计算挂单价格
        bid_price = mid_price * (1 - self.spread/2)
        ask_price = mid_price * (1 + self.spread/2)
        
        # 考虑库存限制调整挂单量
        adj_factor = 1.0
        if self.inventory > self.max_inventory * 0.5:
            # 库存过高，减少买单，增加卖单
            adj_factor = max(0.1, 1 - (self.inventory / self.max_inventory))
        elif self.inventory < -self.max_inventory * 0.5:
            # 库存过低，减少卖单，增加买单
            adj_factor = max(0.1, 1 + (self.inventory / self.max_inventory))
            
        adjusted_amount = self.order_amount * adj_factor
        
        return {
            'bid': {'price': bid_price, 'amount': adjusted_amount},
            'ask': {'price': ask_price, 'amount': adjusted_amount},
            'mid_price': mid_price,
            'timestamp': book_snapshot['timestamp']
        }

    def simulate_order_execution(self, orders, current_trades):
        """模拟订单执行，考虑交易延时"""
        if not orders:
            return
            
        timestamp = orders['timestamp']
        mid_price = orders['mid_price']
        bid_order = orders['bid']
        ask_order = orders['ask']
        
        # 模拟交易延时
        execution_delay = random.uniform(*self.delay_range)
        
        # 寻找延时后的市场价格 (简单模拟)
        # 这里使用延时后的交易数据来估计市场价格变化
        delay_seconds = execution_delay
        target_time = timestamp + timedelta(seconds=delay_seconds)
        
        # 找到最接近目标时间的交易
        time_diff = abs(current_trades['timestamp'] - target_time)
        closest_idx = time_diff.idxmin()
        market_price = current_trades.loc[closest_idx, 'price']
        
        # 检查订单是否会被执行
        bid_executed = market_price <= bid_order['price']
        ask_executed = market_price >= ask_order['price']
        
        # 记录执行的订单
        if bid_executed:
            # 买入执行
            cost = bid_order['price'] * bid_order['amount']
            if self.capital >= cost:
                self.capital -= cost
                self.inventory += bid_order['amount']
                self.executed_orders.append({
                    'type': 'buy',
                    'price': bid_order['price'],
                    'amount': bid_order['amount'],
                    'timestamp': timestamp,
                    'execution_delay': execution_delay,
                    'market_price_at_execution': market_price
                })
        
        if ask_executed:
            # 卖出执行
            if self.inventory >= ask_order['amount']:
                revenue = ask_order['price'] * ask_order['amount']
                self.capital += revenue
                self.inventory -= ask_order['amount']
                self.executed_orders.append({
                    'type': 'sell',
                    'price': ask_order['price'],
                    'amount': ask_order['amount'],
                    'timestamp': timestamp,
                    'execution_delay': execution_delay,
                    'market_price_at_execution': market_price
                })
        
        # 记录当前状态
        self.record_performance(timestamp, mid_price)

    def record_performance(self, timestamp, mid_price):
        """记录绩效数据"""
        # 计算总资产(现金 + 库存价值)
        total_assets = self.capital + self.inventory * mid_price
        self.performance_history.append(total_assets)
        self.inventory_history.append(self.inventory)
        self.capital_history.append(self.capital)
        self.timestamps.append(timestamp)

    def run_simulation(self, start_idx=0, end_idx=None):
        """运行模拟"""
        if self.order_book is None or self.trades is None:
            print("请先加载数据")
            return
            
        # 设置结束索引
        if end_idx is None or end_idx > len(self.order_book):
            end_idx = min(1000, len(self.order_book))  # 限制最大运行步数，避免过慢
            
        print(f"开始模拟 - 从索引 {start_idx} 到 {end_idx}")
        
        # 记录初始状态
        initial_snapshot = self.order_book.iloc[start_idx]
        initial_mid = self.get_mid_price(initial_snapshot)
        self.record_performance(initial_snapshot['timestamp'], initial_mid)
        
        # 逐步运行模拟
        for i in range(start_idx, end_idx):
            # 获取当前订单簿快照
            book_snapshot = self.order_book.iloc[i]
            
            # 生成做市订单
            orders = self.generate_orders(book_snapshot)
            if not orders:
                continue
                
            # 获取当前时间前后的交易数据
            time_window = timedelta(minutes=5)  # 5分钟窗口
            start_time = book_snapshot['timestamp'] - time_window
            end_time = book_snapshot['timestamp'] + time_window
            relevant_trades = self.trades[
                (self.trades['timestamp'] >= start_time) & 
                (self.trades['timestamp'] <= end_time)
            ]
            
            if len(relevant_trades) == 0:
                continue
                
            # 模拟订单执行
            self.simulate_order_execution(orders, relevant_trades)
            
            # 打印进度
            if i % 100 == 0:
                print(f"进度: {i}/{end_idx}, 总资产: {self.performance_history[-1]:.2f}, 库存: {self.inventory:.4f}")
        
        print("模拟完成")
        self.analyze_results()

    def analyze_results(self):
        """分析模拟结果"""
        if not self.performance_history:
            print("没有绩效数据可分析")
            return
            
        # 计算总收益
        initial_assets = self.performance_history[0]
        final_assets = self.performance_history[-1]
        total_return = final_assets - initial_assets
        return_rate = (total_return / initial_assets) * 100
        
        print("\n===== 模拟结果分析 =====")
        print(f"初始资产: {initial_assets:.2f} USDT")
        print(f"最终资产: {final_assets:.2f} USDT")
        print(f"总收益: {total_return:.2f} USDT ({return_rate:.2f}%)")
        print(f"执行订单数量: {len(self.executed_orders)}")
        if self.executed_orders:
            buy_orders = sum(1 for o in self.executed_orders if o['type'] == 'buy')
            sell_orders = len(self.executed_orders) - buy_orders
            print(f"买入订单: {buy_orders}, 卖出订单: {sell_orders}")
        
        # 绘制结果图表
        self.plot_results()

    def plot_results(self):
        """绘制绩效图表"""
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 18))
        
        # 总资产变化
        ax1.plot(self.timestamps, self.performance_history)
        ax1.set_title('总资产变化')
        ax1.set_ylabel('资产 (USDT)')
        ax1.grid(True)
        
        # 库存变化
        ax2.plot(self.timestamps, self.inventory_history)
        ax2.set_title('BTC库存变化')
        ax2.set_ylabel('BTC数量')
        ax2.grid(True)
        
        # 现金变化
        ax3.plot(self.timestamps, self.capital_history)
        ax3.set_title('现金变化')
        ax3.set_ylabel('现金 (USDT)')
        ax3.set_xlabel('时间')
        ax3.grid(True)
        
        plt.tight_layout()
        plt.savefig('market_maker_performance.png')
        print("绩效图表已保存为 market_maker_performance.png")
        plt.show()

if __name__ == "__main__":
    # 创建做市商实例
    mm = MarketMaker(
        initial_capital=100000,
        spread=0.002,  # 0.2%的买卖价差
        order_amount=0.005,  # 每次挂单0.005 BTC
        delay_range=(0.1, 0.8),  # 交易延时0.1-0.8秒
        max_inventory=0.5  # 最大持仓0.5 BTC
    )
    
    # 加载数据
    mm.load_data(
        '../datasets/binance_book_snapshot_25_2020-12-01_BTCUSDT.csv',
        '../datasets/binance_trades_2020-12-01_BTCUSDT.csv'
    )
    
    # 运行模拟
    mm.run_simulation(start_idx=0, end_idx=2000)


加载完成 - 订单簿快照数量: 803306, 交易记录数量: 1994320
开始模拟 - 从索引 0 到 2000
进度: 0/2000, 总资产: 100000.10, 库存: 0.0050
进度: 100/2000, 总资产: 100007.42, 库存: 0.3503
进度: 200/2000, 总资产: 100011.36, 库存: 0.4452
进度: 300/2000, 总资产: 100005.71, 库存: 0.4954
进度: 400/2000, 总资产: 100012.28, 库存: 0.5454
进度: 500/2000, 总资产: 100013.40, 库存: 0.5954
进度: 600/2000, 总资产: 100022.20, 库存: 0.6454
进度: 700/2000, 总资产: 99996.33, 库存: 0.6954
进度: 800/2000, 总资产: 99980.15, 库存: 0.7454
进度: 900/2000, 总资产: 99993.43, 库存: 0.7954
进度: 1000/2000, 总资产: 99985.81, 库存: 0.8454
进度: 1100/2000, 总资产: 99995.61, 库存: 0.8954
进度: 1200/2000, 总资产: 100003.17, 库存: 0.9454
进度: 1300/2000, 总资产: 100010.91, 库存: 0.9954
进度: 1400/2000, 总资产: 99997.89, 库存: 1.0454
进度: 1500/2000, 总资产: 100002.32, 库存: 1.0954
进度: 1600/2000, 总资产: 100016.10, 库存: 1.1454
进度: 1700/2000, 总资产: 100031.72, 库存: 1.1954
进度: 1800/2000, 总资产: 100042.83, 库存: 1.2454
